# Import Required Libraries

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib qt

import pickle

# Visual Function to Compare Input and Output Images

In [3]:
def displayImages(inputImage, outputImage, number):
    inputImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)
    outputImage = cv2.cvtColor(outputImage, cv2.COLOR_BGR2RGB)
    
    # Visualize input vs. pipeline img
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,10))
    fig.subplots_adjust(hspace = .2, wspace = .05)
    
    ax1.imshow(inputImage)
    inputText = "Input Image #" + str(number)
    ax1.set_title(inputText, fontsize = 20)
    
    ax2.imshow(outputImage)
    outputText = "Output Image #" + str(number)
    ax2.set_title(outputText, fontsize = 20)

# Identify and Draw Chessboard Corners

In [4]:
# Lists to store object points and image points
objPoints = [] #Object points store 3D points
imgPoints = []  #Image points store 2D points
    
def chessboardCorners(images): 
    # Prepare object points like (0, 0, 0), (1, 0, 0), (2, 0, 0), (7, 5, 0)...
    objPoints = np.zeros((6*9,3), np.float32)
    objPoints[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    for index, img in enumerate(images):
        img = cv2.imread(fname)
        imgOriginal = img.copy()

        # Converting image(s) to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Finding the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

        # If found, draw corners, add image points, add object points
        if ret == True:
            # Add image points, add object points
            imgPoints.append(corners)
            objPoints.append(objPoints)
            # Draw chessboard corners
            cv2.drawChessboardCorners(img, (9, 6), corners, ret)

In [5]:
dist_pickle = pickle.load(open( "wide_dist_pickle.p", "rb" ))

def cal_undistort(photo, objpoints, imgpoints):
    gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(photo, mtx, dist, None, mtx)
    return undist

Perspective Transform

In [6]:
def perspectiveTransform(img):
    height = img.shape[0]
    width = img.shape[1]
    widthOffset = 300
    heightOffset = 300

    img_size = (img.shape[1], img.shape[0])

    src = np.float32([[575, 464], 
                      [707, 464], 
                      [258, 682], 
                      [1049,682]])

    dst = np.float32([[300, 0], 
                      [950, 0], 
                      [300, 700], 
                      [950, 700]])

    M = cv2.getPerspectiveTransform(src, dst)

    Minv = cv2.getPerspectiveTransform(dst, src)

    output = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)
    return output, Minv

In [7]:
def colorTransformation(image):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    # Grayscale image
    # NOTE: we already saw that standard grayscaling lost color information for the lane lines
    # Explore gradients in other colors spaces / color channels to see what might work better
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
#     color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

In [8]:
def histograms(img):
    bottomHalf = img[img.shape[0]//2:,:]
    histogram = np.sum(bottomHalf, axis = 0)
    return histogram

In [9]:
def find_lane_pixels(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    out_img = np.dstack((img, img, img))
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(img):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    return out_img, left_fit, right_fit, ploty

In [10]:
def measureCurvature(img, left_fit, right_fit, ploty):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

In [11]:
def draw_lane(img, warped, left_fit, right_fit, ploty, left_curverad, right_curverad, Minv):

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    
    # Creating Text and set font parameters
    TextL = "Left  Curvature: " + str(format(left_curverad, ".4f")) + " m"
    TextR = "Right Curvature: " + str(format(right_curverad, ".4f"))+ " m"
    fontScale=1
    thickness=2
    fontFace =cv2.FONT_HERSHEY_SIMPLEX 


    # Using CV2 putText to write text into images
    cv2.putText(newwarp, TextL, (110,60), fontFace, fontScale,(255,255,255), thickness,  lineType = cv2.LINE_AA)
    cv2.putText(newwarp, TextR, (110,110), fontFace, fontScale,(255,255,255), thickness,  lineType = cv2.LINE_AA)

    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    return result

In [27]:
def pipeline(image):
#     images = glob.glob('test_images/*.jpg')
#     for index, image in enumerate(images):
#     img = cv2.imread(image)
    undistorted = cal_undistort(image, dist_pickle["objpoints"], dist_pickle["imgpoints"])
    birdsEyeView, minv = perspectiveTransform(undistorted)
    colorTransformed = colorTransformation(birdsEyeView)
    output, left_fit, right_fit, ploty = fit_polynomial(colorTransformed)
    leftcurve, rightcurve = measureCurvature(output, left_fit, right_fit, ploty)
    lane = draw_lane(image, colorTransformed, left_fit, right_fit, ploty, leftcurve, rightcurve, minv)
    return lane
#         displayImages(img, lane, index + 1)
# pipeline()

In [28]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [33]:
video = 'Output Videos/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video, audio=False)

[MoviePy] >>>> Building video Output Videos/project_video.mp4
[MoviePy] Writing video Output Videos/project_video.mp4


 46%|████▌     | 35/76 [04:17<05:00,  7.32s/it]

KeyboardInterrupt: 

In [32]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video))